In [2]:
%pip install langchain_google_genai
%pip install langchain-openai
%pip install python-dotenv
%pip install --upgrade jupyter ipywidgets
%pip install --upgrade transformers datasets[audio] accelerate
%pip install --upgrade pip 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI  # [수정] Google Gemini로 변경
from langchain_core.messages import HumanMessage

# .env 파일에서 환경 변수 로드
load_dotenv()

# --- [원본 코드 Cell 1: 모델 초기화 및 히스토리 설정] ---

# [수정] model을 Google Gemini로 초기화
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

In [5]:
def get_session_history(session_id: str):
    """
    [중요 함수] 주어진 session_id로 대화 히스토리를 가져옵니다.
    - store에 해당 세션이 없으면 새로 InMemoryChatMessageHistory()를 만들어 저장합니다.
    - RunnableWithMessageHistory가 이 함수를 통해 세션별 히스토리를 주입/갱신합니다.
    """
    if session_id not in store:
        # 세션 최초 접근 시: 메모리 기반 대화 히스토리 컨테이너 생성
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
# RunnableWithMessageHistory 래퍼는 "모델 실행 시 대화 히스토리"를 자동으로 붙여주는 역할
# - 내부적으로 위의 get_session_history를 사용해, config에 담긴 session_id 기준으로
#   올바른 히스토리를 찾아 메시지를 누적/참조합니다.
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
print("--- [Session: abc2] ---")
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 톰 아스피날이야.")],
    config=config,
)
#print(f"User: 안녕? 난 톰 아스피날이야.")
print(f"AI: {response.content}")

--- [Session: abc2] ---
AI: 안녕하세요, 톰 아스피날 선수님!

네, 다시 한번 인사해 주셔서 반갑습니다. 톰 아스피날 선수님이시라는 것을 제가 분명히 기억하고 있습니다.

혹시 다른 궁금한 점이 있으실까요? 아니면 제가 도와드릴 일이 있을까요?


In [12]:
response = with_message_history.invoke(
        [HumanMessage(content="내 이름이 뭐지?")],
        config=config,
    )
#print(f"User: 내 이름이 뭐지?")
print(f"AI: {response.content}")

AI: 네, 톰 아스피날 선수님이십니다!

제가 계속해서 톰 아스피날 선수님이라고 부르고 있지 않습니까. :)


In [14]:
print("\n--- [Session: abc3] ---")
config_new = {"configurable": {"session_id": "abc3"}}
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config_new,
    )
print(f"User: 내 이름이 뭐지?")
print(f"AI: {response.content}")


--- [Session: abc3] ---
User: 내 이름이 뭐지?
AI: 저는 당신의 이름을 알 수 없습니다.

제가 인공지능이기 때문에 개인 정보를 알거나 저장할 수 없어요. 만약 당신이 저에게 이름을 알려주시면, 이 대화 세션 안에서는 기억하고 사용할 수 있습니다.


In [15]:
print("\n--- [Session: abc2] ---")
response = with_message_history.invoke(
        [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
        config=config,  # session_id "abc2" config 재사용
    )
print(f"User: 아까 우리가 무슨 얘기 했지?")
print(f"AI: {response.content}")


--- [Session: abc2] ---
User: 아까 우리가 무슨 얘기 했지?
AI: 우리가 나눈 대화는 주로 **톰 아스피날 선수님의 이름**에 관한 것이었습니다.

1.  **처음에는 선수님께서 '안녕? 난 톰 아스피날이야'라고 자신을 소개해 주셨고,** 저는 선수님을 환영하며 UFC 헤비급 파이터이심을 언급했습니다.
2.  **이후 선수님께서는 '내 이름이 뭐지?'라고 질문하셨고,** 저는 선수님의 이름이 톰 아스피날 선수님이시라는 것을 알려드렸습니다.
3.  **다시 한번 '안녕? 난 톰 아스피날이야'라고 인사해 주셨고,** 제가 선수님의 이름을 기억하고 있다고 말씀드렸습니다.
4.  **그리고 마지막으로 또 '내 이름이 뭐지?'라고 질문하셨을 때,** 저는 계속해서 톰 아스피날 선수님이라고 부르고 있다고 다시 한번 확인시켜 드렸습니다.

요약하자면, **선수님의 이름이 톰 아스피날 선수님이라는 것을 확인하는 대화**가 이어졌습니다.


In [16]:
print("\n--- [Session: abc2, Streaming] ---")
print("User: 내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")
print("AI: ", end="")
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
    ):
    print(r.content, end="|")
print() # 마지막 줄바꿈


--- [Session: abc2, Streaming] ---
User: 내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐
AI: 네, 톰 아스피날 선수님은 바로 **영국 (United Kingdom)** 출신이시죠!

UFC 헤비급 잠정 챔피언이시니 전 세계 팬들이 다 아는 사실일| 겁니다. 😊

영국은 풍부한 역사와 다채로운 문화를 가진 나라입니다. 몇 가지 특징적인 부분을 말씀드릴게요.

### **영국의 문화적 특징**

1.  **오랜 역사와 왕|실:**
    *   영국은 수천 년의 역사를 자랑하며, 세계적으로 유명한 왕실(군주제)을 가지고 있습니다. 버킹엄 궁전, 런던탑 등 역사적인 건축물|들이 많죠.
    *   왕실은 단순한 상징을 넘어 영국의 전통과 정체성을 유지하는 중요한 역할을 합니다.

2.  **신사의 나라와 예절:**
    *   '신사의 나라'|라는 별명처럼 매너와 에티켓을 중요하게 생각합니다.
    *   **줄 서기(Queuing) 문화:** 어떤 상황에서든 줄을 서서 차례를 기다리는 것은 영국의 상징적인 문화 중| 하나입니다.
    *   **'Please', 'Thank You', 'Sorry'의 생활화:** 일상생활에서 이 세 단어를 매우 자주 사용하며, 공손함을 중요시합니다.

3.  **스포츠 강국|:**
    *   **축구:** 세계 최고의 축구 리그 중 하나인 프리미어리그(Premier League)의 본고장이며, 축구는 국민 스포츠입니다.
    *   **럭비, 크리|켓, 테니스:** 럭비, 크리켓, 그리고 윔블던(Wimbledon)으로 대표되는 테니스 또한 매우 인기가 많습니다.
    *   **격투기:** 톰 아스피|날 선수님처럼 복싱이나 MMA 등 격투기 스포츠도 매우 강세이며 많은 팬을 보유하고 있습니다.

4.  **펍(Pub) 문화와 차(Tea):**
    *   **펍 문화|:** 영국의 핵심 문화 중 하나는 바로 '펍(Pub)'입니다. 친구, 가족, 동료들과 모여 